# **Chatbot:**

In [1]:
!pip install datasets
!pip install datasets transformers
!pip install datasets transformers peft
!pip install transformers tensorflow
!pip install langchain transformers datasets peft tensorflow
!pip install streamlit
!pip install gtts
!pip install pyngrok
!pip install datasets transformers peft
!pip install langchain transformers datasets peft tensorflow
!pip install langchain-community
!pip install bitsandbytes
!pip install whisper
!pip install git+https://github.com/openai/whisper.git
!pip install streamlit soundfile
!pip install audio-recorder-streamlit
!pip install fpdf
# !pip install autoawq

  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 60.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 98.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 30.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:

In [4]:
# !pip show autoawq

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-as_pfl0z
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-as_pfl0z
  Resolved https://github.com/openai/whisper.git to commit 5979f03701209bb035a0a466f14131aeb1116cbb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803559 sha256=4913174097dac8d764b72ac76388c9a9d161012425a56f575ae87de002b90b53
  Stored in directory: /tmp/pip-ephem-wheel-cache-vsjqukqc/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 46.0 MB/s et

In [ ]:
%%writefile streamlit_app3.py

import streamlit as st
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TFAutoModelForSequenceClassification
from langchain.prompts import ChatPromptTemplate
from transformers import AutoModel, AutoTokenizer
from langchain import LLMChain, PromptTemplate, HuggingFacePipeline
from langchain.memory.buffer import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
from peft import PeftModel, LoraConfig, get_peft_model
from datasets import Dataset
from audio_recorder_streamlit import audio_recorder
import pandas as pd
import whisper
import tempfile
import torch
from gtts import gTTS
import os
import streamlit as st
import soundfile as sf
import numpy as np
from io import BytesIO
from fpdf import FPDF

# Custom CSS styling
st.markdown("""
    <style>
    body {
        background-size: cover;
    }
    .main {
        background: radial-gradient(circle at 10% 20%, rgb(69, 86, 102) 0%, rgb(34, 34, 34) 90%);
        padding: 20px;
        border-radius: 10px;
        box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
    }
    .message {
        padding: 10px;
        border-radius: 10px;
        margin-bottom: 10px;
    }
    .user {
        background-color: #6897ab;
    }
    .assistant {
        background-color: #848a86;
        color: white;
    }
    </style>
""", unsafe_allow_html=True)


# Load Whisper model
@st.cache_resource
def load_whisper_model():
    return whisper.load_model("base")  # You can switch to "small", "medium", or "large" models.

# Whisper transcription function
def transcribe_audio(audio_file):
    try:
        result = whisper_model.transcribe(audio_file.name)
        return result["text"]
    except Exception as e:
        st.error(f"Error transcribing audio: {e}")
        return ""


class StoryCreativityChain:
    def __init__(self, model, tokenizer):
        self.llmPipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            torch_dtype=torch.float16,
            device_map="auto",
            max_new_tokens=500,
            do_sample=True,
            top_k=30,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )
        self.llm = HuggingFacePipeline(pipeline=self.llmPipeline, model_kwargs={'temperature': 0.7, 'max_length': 5, 'top_k': 50})

    def getPromptFromTemplate(self):
        system_prompt = """
        Role: You are a medical assistant chatbot designed to gather a patient's initial data before they consult with a doctor. You are not allowed to provide diagnoses, medical advice, or information outside the medical domain. Your purpose is to ask structured questions to collect comprehensive health-related details and then guide the patient to generate an initial report for the doctor.

        Objectives:
        Your primary task is to collect relevant medical information in a conversational manner while staying strictly within the context of gathering patient data. You must:
            1. Collect the patient's medical history, symptoms, lifestyle habits, daily routine, diet plan, and exercise routine.
            2. Use adaptive questioning to explore details about specific symptoms the patient mentions.
            3. Avoid repeating questions already answered by the user.
            4. Conclude the conversation once all relevant data is collected.
        
        Guidelines:
        
            1. Question Flow:
        
                Start with basic details like name, age, and gender. Don't ask these details if user has already told.
                Avoid repeating questions already answered by the user like name, age, gender, etc.
                
                Progress to questions about the patient's symptoms:
                    "Can you describe your symptoms?"
                    "When did these symptoms begin?"
                    "How severe are the symptoms on a scale of 1 to 10?"
                    
                After symptoms, move to medical history:
                    "Have you had any major illnesses or surgeries in the past?"
                    "Do you have any chronic conditions like diabetes or hypertension?"
                    
                Next, inquire about daily habits and routines:
                    "What does your daily diet look like?"
                    "Do you exercise regularly? If yes, what kind of exercise?"
                    "Do you smoke, drink alcohol, or have other habits affecting your health?"
                
                
            2. Adaptivity: Tailor questions based on the patient's responses. For instance:
        
                If a patient mentions chest pain, ask:
                    "When did the chest pain start?"
                    "Is the pain constant or does it come and go?"
                    "Do you have any other symptoms, like shortness of breath or dizziness?"
                    
            3. Context Adherence:
                
                    Do not discuss unrelated topics, including non-medical questions or tasks like coding, general knowledge, or advice unrelated to patient data gathering.
                    If the patient asks unrelated or out-of-scope questions, respond politely:
                        "I am here to assist with gathering your initial health information for the doctor. For other queries, I recommend consulting an appropriate expert."
        
            4. Ending the Conversation:
        
                Once sufficient data is collected, conclude with a polite statement:
                    "I have gathered your initial data. Now you can generate your initial report to share with the doctor before consultation. Take care!"
                Do not continue the conversation unnecessarily.

            5. Avoid repetition: If a user provides information (e.g., name, age, or symptom duration), acknowledge it without asking again.

                
        Key Interaction Flow:
        
            1. Greet the patient and explain your purpose.
            
                "Hello! I am here to gather your initial health information before your consultation with the doctor. May I start by asking a few questions?"
        
            2. Follow the structured questioning sequence:
            
                Basic details → Symptoms → Medical history → Daily routine → Lifestyle habits → End.
        
            3. Conclude the conversation after collecting all relevant information.
            
        
        Response Control:
        
            1. Keep responses concise (under 100 words).
            2. Always stay on topic and avoid repetition.
            3. Ensure privacy, professionalism, and empathy.

        """
        
        B_INST, E_INST = "[INST]", "[/INST]"
        B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
        SYSTEM_PROMPT1 = B_SYS + system_prompt + E_SYS

        instruction = """
            History: {history} \n
            User: {question}"""
        
        prompt_template = B_INST + SYSTEM_PROMPT1 + instruction + E_INST
        prompt = PromptTemplate(input_variables=["history", "question"], template=prompt_template)

        return prompt

    def clean_response(self, response):
        # Remove any extraneous tags like "AI:" and trim responses before "Human:" or "human:"
        response = response.replace("AI:", "").strip()
        split_response = response.split("Human:")[0].split("human:")[0].split("User:")[0].split("user:")[0]
        return split_response.strip()
    
    def getNewChain(self):
        prompt = self.getPromptFromTemplate()
        memory = ConversationBufferMemory(input_key="question", memory_key="history", max_len=5)

        # Initialize LLMChain with proper parameters
        llm_chain = LLMChain(prompt=prompt, llm=self.llm, verbose=True, memory=memory, output_parser=CustomOutputParser())

        # Return a callable that processes inputs using the chain
        def run_chain(inputs):
            question = inputs.get("question", "")
            response = llm_chain.run({"history": "", "question": question})
            # return response["text"] if 'text' in response else response
            return self.clean_response(response)

        return run_chain

class CustomOutputParser(StrOutputParser):
    def parse(self, response: str):
        return response.split('[/INST]')[-1].strip()

# Cache the model and tokenizer to load them only once
@st.cache_resource
def load_model_and_tokenizer():
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto"
    )

    return model, tokenizer


# Generate PDF from conversation history
def generate_pdf(conversation):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Healthcare Initial Report", ln=True, align="C")
    pdf.ln(10)
    for message in conversation:
        role = "User" if message["role"] == "user" else "Assistant"
        pdf.multi_cell(0, 10, f"{role}: {message['content']}")
        pdf.ln(2)
    pdf_path = "healthcare_initial_report.pdf"
    pdf.output(pdf_path)
    return pdf_path
    

# Load models
whisper_model = load_whisper_model()
model, tokenizer = load_model_and_tokenizer()

# Initialize the chatbot
story_chain = StoryCreativityChain(model, tokenizer)
if "chain" not in st.session_state:
    st.session_state.chain = story_chain.getNewChain()
    st.session_state.history = []

# Function to convert text to audio
def text_to_audio(text, filename="response.mp3"):
    try:
        tts = gTTS(text=text)
        tts.save(filename)
        return filename
    except Exception as e:
        st.error(f"Error generating audio: {e}")
        return None

# Function to transcribe audio using Whisper
def transcribe_audio(audio_file):
    try:
        transcription = whisper_model.transcribe(audio_file.name)
        return transcription["text"]
    except Exception as e:
        st.error(f"Error transcribing audio: {e}")
        return ""

# Streamlit UI
st.title("Welcome to the Healthcare Assistant Chatbot!")
st.sidebar.title("Healthcare Assistant")
st.sidebar.write("Use text or voice to communicate with the assistant.")

# Record and transcribe audio input
audio_bytes = audio_recorder()
if audio_bytes:
    st.audio(audio_bytes, format="audio/wav")
    # Create a temporary file to save the audio bytes
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_audio:
        temp_audio.write(audio_bytes)  # Directly write audio_bytes
        temp_audio_path = temp_audio  # Get the temporary file path
    transcribed_text = transcribe_audio(temp_audio_path)  # Pass the file path to the transcription function
    st.write(f"**Transcribed:** {transcribed_text}")
else:
    transcribed_text = ""

    
# User input via text or voice transcription
user_input = st.text_input("Enter your prompt:", value=transcribed_text)

if st.button("Generate Response"):
    if user_input.strip():
        st.session_state.history.append({"role": "user", "content": user_input})
        response = st.session_state.chain({"question": user_input})
        audio_file = text_to_audio(response)
        st.session_state.history.append({"role": "assistant", "content": response})
        st.audio(audio_file)
    else:
        st.warning("Please enter a valid prompt.")


if st.button("Generate Report"):
    report_input = "Generate an accurate and complete report based on our complete conversation."
    report_response = st.session_state.chain({"question": report_input})
    st.session_state.history.append({"role": "assistant", "content": report_response})
    pdf_path = generate_pdf(st.session_state.history)
    with open(pdf_path, "rb") as pdf_file:
        st.download_button(
            label="Download Report as PDF",
            data=pdf_file,
            file_name="healthcare_report.pdf",
            mime="application/pdf"
        )

# Display chat history
st.write('<div class="main">', unsafe_allow_html=True)
for message in st.session_state.history:
    role_class = "user" if message["role"] == "user" else "assistant"
    st.write(f'<div class="message {role_class}">{message["content"]}</div>', unsafe_allow_html=True)
st.write('</div>', unsafe_allow_html=True)

Overwriting streamlit_app3.py


In [4]:
!ngrok authtoken 2jb86JKOHKjdCaoHKnPIeqPbJJ9_6Zrx6tHZQeLacbEwNFuCQ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [5]:
from pyngrok import ngrok

# Start a new ngrok tunnel
public_url = ngrok.connect(8000)
print("Ngrok public URL:", public_url)

Ngrok public URL: NgrokTunnel: "https://b284-35-229-234-174.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
!streamlit run streamlit_app3.py --server.port 8000




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8000
  Network URL: http://172.19.2.2:8000
  External URL: http://35.229.234.174:8000

2025-03-02 20:28:46.814977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-02 20:28:47.018506: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-02 20:28:47.077873: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:30: UserWarning: Importing LLMChain from langchain root module is no longer supported. Please use langchain.chains.LLMChain ins

In [2]:
from huggingface_hub import login

login(token="")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
